In [ ]:
# - talk to BigQuery (google-cloud-bigquery)
# - use Vertex AI models (vertexai)
# - work with data frames (pandas)

!pip -q install google-cloud-bigquery vertexai pandas

In [ ]:
# I have imported the needed libraries
import os
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel
from google.cloud import bigquery

PROJECT_ID: qwiklabs-gcp-02-0ee695e6ee11
GCS_URI: gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv


In [ ]:

# I have pulled the active Google Cloud project id from the environment.
# In hosted notebook environments, this is usually already set.

PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]
REGION = "us"

# BigQuery dataset + table names I’ll create/use
DATASET = "AuroraBay"
RAW_TABLE = "aurora_bay_faqs"
EMBED_TABLE = "aurora_bay_faqs_embedded"

GCS_URI = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"


# This client is my connection to BigQuery (so I can run SQL)
bq = bigquery.Client(project=PROJECT_ID)

# Quick prints so I can confirm configuration is correct
print("PROJECT_ID:", PROJECT_ID)
print("GCS_URI:", GCS_URI)

In [ ]:
# I have written a SQL script that:
# 1) creates the dataset if it doesn't exist
# 2) loads the CSV from GCS into a BigQuery table

sql = f"""
-- Create dataset if needed
CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{DATASET}`;

-- Load CSV from GCS into a native BigQuery table
LOAD DATA OVERWRITE `{PROJECT_ID}.{DATASET}.{RAW_TABLE}`
(
  question STRING,
  answer STRING
)
FROM FILES (
  format = 'CSV',
  uris = ['{GCS_URI}'],
  skip_leading_rows = 1
);
"""

# .result() forces the query to finish before moving on
bq.query(sql).result()
print("Loaded FAQs into BigQuery:", f"{PROJECT_ID}.{DATASET}.{RAW_TABLE}")

Loaded FAQs into BigQuery: qwiklabs-gcp-02-0ee695e6ee11.AuroraBay.aurora_bay_faqs


In [ ]:
# I run a quick SELECT to make sure the data loaded correctly.
# Converting to dataframe makes it easy to view inside the notebook.

bq.query(f"SELECT * FROM `{PROJECT_ID}.{DATASET}.{RAW_TABLE}` LIMIT 5;").to_dataframe()

,question,answer
0,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...
1,What is the population of Aurora Bay?,Aurora Bay has a population of approximately 3...
2,Where is the Aurora Bay Town Hall located?,The Town Hall is located at 100 Harbor View Ro...
3,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ..."
4,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...


In [ ]:
# This is the BigQuery connection name that points to Vertex AI.
# You may need to change it if your environment uses a different connection.

CONNECTION = "us.embedding_conn"  

# I have created a BigQuery ML REMOTE model:
# - "REMOTE" means BigQuery calls Vertex AI behind the scenes.
# - ENDPOINT is the embedding model name in Vertex AI.  

sql = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET}.Embeddings`
REMOTE WITH CONNECTION `{CONNECTION}`
OPTIONS (ENDPOINT = 'text-embedding-004');
"""
bq.query(sql).result()
print("Created remote embedding model.")

Created remote embedding model.


In [ ]:
#  Here I have created a new table where each row includes:
# - question
# - answer
# - content (combined text)
# - embedding vector (ml_generate_embedding_result)
# - stats (ml_generate_embedding_statistics)

# Important idea:
# Embeddings work better when I embed a richer combined string (Q + A),
# not just the question alone.


sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.{EMBED_TABLE}` AS
SELECT
  question,
  answer,
  CONCAT('QUESTION: ', question, '\\nANSWER: ', answer) AS content,
  ml_generate_embedding_result,
  ml_generate_embedding_statistics
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET}.Embeddings`,
  (
    SELECT
      question,
      answer,
      CONCAT('QUESTION: ', question, '\\nANSWER: ', answer) AS content
    FROM `{PROJECT_ID}.{DATASET}.{RAW_TABLE}`
  )
);
"""
bq.query(sql).result()
print("Created embedded table:", f"{PROJECT_ID}.{DATASET}.{EMBED_TABLE}")

Created embedded table: qwiklabs-gcp-02-0ee695e6ee11.AuroraBay.aurora_bay_faqs_embedded


In [ ]:

# I have checked the length of the embedding vector to confirm it was generated.
# (All rows should have the same embedding_dim.)

bq.query(f"""
SELECT
  ARRAY_LENGTH(ml_generate_embedding_result) AS embedding_dim
FROM `{PROJECT_ID}.{DATASET}.{EMBED_TABLE}`
LIMIT 1;
""").to_dataframe()

,embedding_dim
0,768


In [ ]:

# This cell recreates the embedding table but removes the statistics column.
# That makes the table smaller and simpler for vector search.


sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.{EMBED_TABLE}` AS
SELECT
  question,
  answer,
  CONCAT('QUESTION: ', question, '\\nANSWER: ', answer) AS content,
  ml_generate_embedding_result
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET}.Embeddings`,
  (
    SELECT
      question,
      answer,
      CONCAT('QUESTION: ', question, '\\nANSWER: ', answer) AS content
    FROM `{PROJECT_ID}.{DATASET}.{RAW_TABLE}`
  )
);
"""
bq.query(sql).result()
print("Rebuilt embedded table with question/answer/content + embedding.")

Rebuilt embedded table with question/answer/content + embedding.


In [ ]:
# I embed a test query ("test") and run VECTOR_SEARCH.
# This verifies my embedded table can be searched successfully.

sql = f"""
WITH query_vec AS (
  SELECT ml_generate_embedding_result AS q_emb
  FROM ML.GENERATE_EMBEDDING(
    MODEL `{PROJECT_ID}.{DATASET}.Embeddings`,
    (SELECT 'test' AS content)
  )
)
SELECT *
FROM VECTOR_SEARCH(
  TABLE `{PROJECT_ID}.{DATASET}.{EMBED_TABLE}`,
  'ml_generate_embedding_result',
  (SELECT q_emb FROM query_vec),
  top_k => 1
) AS vs;
"""
bq.query(sql).to_dataframe()

,query,base,distance
0,"{'q_emb': [0.026759734377264977, 0.01896281354...",{'question': 'How are local schools performing...,1.159976


In [ ]:
# Make vector search reusable (fn)

def bq_vector_search(user_question: str, top_k: int = 5):
    sql = f"""
    WITH query_vec AS (
      SELECT ml_generate_embedding_result AS q_emb
      FROM ML.GENERATE_EMBEDDING(
        MODEL `{PROJECT_ID}.{DATASET}.Embeddings`,
        (SELECT @q AS content)
      )
    )
    SELECT
      vs.base.question AS question,
      vs.base.answer   AS answer,
      vs.base.content  AS content
    FROM VECTOR_SEARCH(
      TABLE `{PROJECT_ID}.{DATASET}.{EMBED_TABLE}`,
      'ml_generate_embedding_result',
      (SELECT q_emb FROM query_vec),
      top_k => @top_k
    ) AS vs;
    """

    job = bq.query(
        sql,
        job_config=bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("q", "STRING", user_question),
                bigquery.ScalarQueryParameter("top_k", "INT64", top_k),
            ]
        ),
    )
    return [dict(r) for r in job.result()]

# Test
def retrieve_answers_only(user_question: str, top_k: int = 3):
    matches = bq_vector_search(user_question, top_k=top_k)
    return [m["answer"] for m in matches]

retrieve_answers_only("things to do in aurora", top_k=1)

['Popular activities include fishing, kayaking, hiking in the nearby forests, and northern lights viewing in the winter.']

In [ ]:
# # ------------------------------------------------------------
# # Simple RAG Chatbot Loop (BigQuery Vector Search + Gemini)
# # This cell turns our retrieval function into a chatbot:
# #   1) user asks a question
# #   2) we retrieve top-k relevant FAQ rows via VECTOR_SEARCH
# #   3) we pass the retrieved context + question to Gemini
# #   4) we loop until user types "done"
# # ------------------------------------------------------------
# import vertexai
# from vertexai.generative_models import GenerativeModel

# # Initialize Vertex AI (uses your project + region)
# vertexai.init(project=PROJECT_ID, location="us-central1")

# # Choose a Gemini model available in Vertex AI
# # If your environment requires a specific model name, this is the one most commonly used:
gemini = GenerativeModel( "gemini-2.5-flash")

def build_context(matches, max_chars=6000):
    """
    Converts retrieved FAQ matches into a compact context string for Gemini.
    max_chars prevents sending too much text to the model.
    """
    parts = []
    for i, m in enumerate(matches, start=1):
        parts.append(f"[FAQ {i}] Q: {m['question']}\nA: {m['answer']}")
    context = "\n\n".join(parts)
    return context[:max_chars]

def rag_answer(user_question: str, top_k: int = 5) -> str:
    """
    1) Retrieves relevant FAQ entries from BigQuery using vector search
    2) Sends them to Gemini with the user's question
    3) Returns a grounded answer (based only on the provided FAQ context)
    """
    matches = bq_vector_search(user_question, top_k=1)
    context = build_context(matches)

    prompt = f"""
You are an assistant for the town of Aurora Bay, Alaska.
Answer the user's question using ONLY the FAQ context provided.
If the answer is not in the context, say you don't have enough information.

FAQ Context:
{context}

User Question:
{user_question}

Answer:
""".strip()

    response = gemini.generate_content(prompt)
    return response.text

print("Aurora Bay Chatbot is ready ✅")
print("Type your question. Type 'done' to exit.\n")

while True:
    user_q = input("You: ").strip()
    if user_q.lower() in {"done", "exit", "quit"}:
        print("Bot: Done. Goodbye! 👋")
        break
    if not user_q:
        print("Bot: Please type a question (or 'done' to exit).")
        continue

    try:
        answer = rag_answer(user_q, top_k=2)
        print(f"\nBot: {answer}\n")
    except Exception as e:
        print(f"\nBot: I hit an error: {e}\n")



Aurora Bay Chatbot is ready ✅
Type your question. Type 'done' to exit.

You: where to park in aurora

Bot: I don't have enough information.

You: things to do in aurora

Bot: Popular activities include fishing, kayaking, hiking in the nearby forests, and northern lights viewing in the winter.

You: done
Bot: Done. Goodbye! 👋
